In [2]:
import pandas as pd

df = pd.read_csv("../data/processed/master_uniprots.csv")

In [25]:
orphan_names = []
for r in df[df["strain"] != "H37Rv"].iterrows():
    orphan_names += [(r[1]["name"], r[1]["uniprot_id"])]

In [26]:
uniprot_outs = []
chembl_uniprot = {}
for n in orphan_names:
    if str(n[0]) != "nan":
        df_ = df[df["name"] == n[0]]
        uniprot_in = df_[df_["strain"] == "H37Rv"]["uniprot_id"].tolist()
        uniprot_out = df_[df_["strain"] != "H37Rv"]["uniprot_id"].tolist()
        in_chembl = min(1, sum(df_["in_chembl"].tolist()))
        uniprot_outs += uniprot_out
        for p in uniprot_in:
            chembl_uniprot[p] = in_chembl
    else:
        uniprot_outs += [n[1]]


In [24]:
df[df["uniprot_id"].isin(uniprot_outs)]

,ORFID,name,patric,essentiality,strain,VI,VI_lower,VI_higher,high_confidence,uniprot_id,in_chembl,in_known
3938,NaN,pncA,Nicotinamide deamidase,NaN,NaN,NaN,NaN,NaN,NaN,Q50575,1,0
3939,NaN,ptbB,Phosphotyrosine protein phosphatase ptpB,NaN,NaN,NaN,NaN,NaN,NaN,A0A0H3M0T0,1,0
3940,MRA_3015,hupB,DNA-binding protein HupB,NaN,Hr7Ra,NaN,NaN,NaN,NaN,A5U6Z7,1,0
3941,MT0483,icl1,Isocitrate lyase 1,NaN,CDC1551,NaN,NaN,NaN,NaN,P9WKK6,1,0
3942,NaN,iphP,Phosphotyrosine protein phosphatase,NaN,NaN,NaN,NaN,NaN,NaN,A0A045ISB3,1,0
3943,NaN,birA,biotin--[biotin carboxyl-carrier protein] ligase,NaN,NaN,NaN,NaN,NaN,NaN,P96884,1,0
3944,MRA_2218,adoK,Adenosine kinase,NaN,Hr7Ra,NaN,NaN,NaN,NaN,A5U4N0,1,0
3945,MT1988,NaN,Epoxide hydrolase B,NaN,CDC1551,NaN,NaN,NaN,NaN,P95276,1,0
3946,MT1345,atpE,ATP synthase subunit c,NaN,CDC1551,NaN,NaN,NaN,NaN,P9WPS0,1,0
3947,MT3571,rmlC,"dTDP-4-dehydrorhamnose 3,5-epimerase",NaN,CDC1551,NaN,NaN,NaN,NaN,P9WH10,1,0
